In [1]:
import os

In [4]:
%pwd

'/home/asus/Documents/Chest-Cancer-Classification/Project'

In [3]:
cd ../

/home/asus/Documents/Chest-Cancer-Classification/Project


In [12]:
os.chdir("../")

/home/asus/Documents/Chest-Cancer-Classification/Project


## Update the entity

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [9]:
from src.cnnClassifier.constants import *
from  src.cnnClassifier.utils.common import read_yaml , create_directories

## Update the configuration

In [11]:


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      



In [13]:
import os
import zipfile
import gdown
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

## Update the components

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


## Update the pipeline

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-28 12:34:37,325: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-28 12:34:37,333: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-28 12:34:37,336: INFO: common: created directory at: artifacts]
[2024-08-28 12:34:37,339: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-28 12:34:37,341: INFO: 549056089: Downloading data from https://drive.google.com/file/d/1K_-ObVEp_YhvNSGAlFRiZG6KU1e-d6uL/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1K_-ObVEp_YhvNSGAlFRiZG6KU1e-d6uL
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1K_-ObVEp_YhvNSGAlFRiZG6KU1e-d6uL&confirm=t&uuid=91f5d480-1b0b-41f8-a2e2-a1ceb92b6e76
To: /home/asus/Documents/Chest-Cancer-Classification/Project/artifacts/data_ingestion/data.zip
100%|██████████| 49.0M/49.0M [00:19<00:00, 2.57MB/s]

[2024-08-28 12:35:01,722: INFO: 549056089: Downloaded data from https://drive.google.com/file/d/1K_-ObVEp_YhvNSGAlFRiZG6KU1e-d6uL/view?usp=sharing into file artifacts/data_ingestion/data.zip]
